# COVbat 5 way TOP and StrokeMRI  vs. EDIS , Insight, Helius and SABRE harmonized datasets

Note this must be run in the `covbat` environment. Environment file inside vendor/covbat

## import libraries

In [ ]:
import os
import sys

import pandas as pd
import numpy as np

import patsy

sys.path.insert(0, '../../') # path to functions

import cvasl.harmony as har
import cvasl.vendor.covbat.covbat as covbat

import data

In [ ]:
EDIS = pd.read_csv('../new_data/TrainingDataComplete_EDIS.csv')
HELIUS = pd.read_csv('../new_data/TrainingDataComplete_HELIUS.csv')
SABRE = pd.read_csv('../new_data/TrainingDataComplete_SABRE.csv')
MRI = pd.read_csv('../new_data/TrainingDataComplete_StrokeMRI.csv')
TOP = pd.read_csv('../new_data/TrainingDataComplete_TOP.csv')
Insight46 = pd.read_csv('../new_data/TrainingDataComplete_Insight46.csv')

In [ ]:
#HELIUS

## We found there is one participant from two seperate sights (HELIUS and SABRE) named the same. There fore we will show and switch this

In [ ]:
HELIUS[HELIUS['participant_id']=='sub-153852_1']

In [ ]:
SABRE[SABRE['participant_id']=='sub-153852_1']

In [ ]:
HELIUS.loc[HELIUS['participant_id']=='sub-153852_1', 'participant_id'] = 'sub-153852_1H'

In [ ]:
EDIS.head(3)

In [ ]:
TOP = TOP.drop([ 'ID'], axis= 1)

TOP.head(3)

In [ ]:
MRI = MRI.drop(['ID'],axis = 1)
MRI.tail(5)

In [ ]:
EDIS = EDIS.drop(['ID'],axis = 1)
EDIS['Site'] = 1
EDIS.tail(5)

In [ ]:
HELIUS = HELIUS.drop(['ID'],axis = 1)
HELIUS['Site'] = 2
HELIUS.tail(5)

In [ ]:
SABRE = SABRE.drop(['ID'],axis = 1)
SABRE['Site'] = 3
SABRE.tail(3)

In [ ]:
Insight46 = Insight46.drop(['ID'],axis = 1)
Insight46['Site'] = 4
Insight46.tail(5)

In [ ]:
TOPMRI = pd.concat([TOP, MRI])
TOPMRI['Site'] = 0
TOPMRI.head(3) 

In [ ]:
TOPMRI.tail(3)

In [ ]:
to_be_harmonized_or_covar = [
    'Age', 'Sex', 'ACA_B_CoV', 'MCA_B_CoV', 'PCA_B_CoV','DeepWM_B_CoV',
     'ACA_B_CBF', 'MCA_B_CBF', 'PCA_B_CBF', 'LD', 'PLD','DeepWM_B_CBF',
       'Labelling', 'Readout', 'M0','TotalGM_B_CoV',
       'TotalGM_B_CBF',
]
not_harmonized= ['GM_vol', 'WM_vol', 'CSF_vol','GM_ICVRatio', 'GMWM_ICVRatio', 'WMHvol_WMvol', 'WMH_count',
                'LD', 'PLD', 'Labelling',
       'Readout', 'M0','DeepWM_B_CoV','DeepWM_B_CBF',]
TOPMRI_semi_features = TOPMRI.drop(to_be_harmonized_or_covar,axis=1)
SABRE_semi_features = SABRE.drop(to_be_harmonized_or_covar,axis=1) 
EDIS_semi_features = EDIS.drop(to_be_harmonized_or_covar,axis=1)
HELIUS_semi_features = HELIUS.drop(to_be_harmonized_or_covar,axis=1) 
Insight46_semi_features = Insight46.drop(to_be_harmonized_or_covar,axis=1) 

TOPMRI = TOPMRI.drop(not_harmonized,axis=1)
SABRE = SABRE.drop(not_harmonized,axis=1) 
EDIS = EDIS.drop(not_harmonized,axis=1) 
HELIUS = HELIUS.drop(not_harmonized,axis=1) 
Insight46 = Insight46.drop(not_harmonized,axis=1) 

In [ ]:
datasets = [TOPMRI, EDIS, SABRE, HELIUS, Insight46]

In [ ]:
ALLFIVE = pd.concat(datasets)

In [ ]:
ALLFIVE

In [ ]:
phenoALLFIVE = ALLFIVE[['participant_id','Age', 'Sex', 'Site']]
phenoALLFIVE = phenoALLFIVE.set_index('participant_id')
print(len(phenoALLFIVE))
phenoALLFIVE.head(3)

In [ ]:
dat_ALLFIVE = ALLFIVE.set_index('participant_id')
print(len(dat_ALLFIVE))
dat_ALLFIVE = dat_ALLFIVE.T 

In [ ]:
dat_ALLFIVE.head(3)

In [ ]:
print(len(phenoALLFIVE))
phenoALLFIVE.head(3)

In [ ]:
phenoALLFIVE.isna().sum()

In [ ]:
modZ = patsy.dmatrix("~ Age + Sex", phenoALLFIVE, return_type="dataframe")
print(len(modZ))
modZ.head(3)


In [ ]:
dat_ALLFIVE.head(2)


In [ ]:
#dat_ALLFIVE.tail(17)

In [ ]:
phenoALLFIVE['Site']

In [ ]:
modZ.head(3)

In [ ]:
#modZ.isna().sum()
dat_ALLFIVE.tail(10)

In [ ]:
modZ.shape

In [ ]:
covbatALLFIVE = covbat.combat(dat_ALLFIVE.tail(10), phenoALLFIVE['Site'],  model=modZ, numerical_covariates ="Age")

In [ ]:
dat_ALLFIVE.head(3)

In [ ]:
covbatALLFIVE = covbatALLFIVE[2:]
covbatALLFIVE.head(3)

In [ ]:
covbatALLFIVE = pd.concat([dat_ALLFIVE.head(3), covbatALLFIVE])
covbatALLFIVE = covbatALLFIVE.T
covbatALLFIVE = covbatALLFIVE.reset_index()

In [ ]:
covbatALLFIVE

In [ ]:
TOPMRI_adjusted = covbatALLFIVE[covbatALLFIVE['Site'] == 0]
EDIS_adjusted   = covbatALLFIVE[covbatALLFIVE['Site'] == 1] 
HELIUS_adjusted = covbatALLFIVE[covbatALLFIVE['Site'] == 2] 
SABRE_adjusted  = covbatALLFIVE[covbatALLFIVE['Site'] == 3] 
Insight_adjusted= covbatALLFIVE[covbatALLFIVE['Site'] == 4] 

In [ ]:
# sites = [TOPMRI_adjusted, 
#          EDIS_adjusted,
#          HELIUS_adjusted, 
#          SABRE_adjusted,
#          Insight_adjusted]
# for site in sites:
#     #print(site)
#     site = site.drop('Site', axis=1)
#     print(site)

    

In [ ]:
TOPMRI_adjusted.head(3)

In [ ]:
TOPMRI_adjusted = TOPMRI_adjusted.drop('Site', axis=1)
TOPMRI_adjusted.head(3)

In [ ]:
EDIS_adjusted = EDIS_adjusted.drop('Site', axis=1)
EDIS_adjusted.tail(3)

In [ ]:
SABRE_adjusted = SABRE_adjusted.drop('Site', axis=1)
SABRE_adjusted.tail(3)

In [ ]:
HELIUS_adjusted = HELIUS_adjusted.drop('Site', axis=1)
HELIUS_adjusted.tail(3)

In [ ]:
Insight_adjusted = Insight_adjusted.drop('Site', axis=1)
Insight_adjusted.tail(3)

In [ ]:
TOPMRI_adjusted = TOPMRI_adjusted.merge(TOPMRI_semi_features,on='participant_id')
EDIS_adjusted = EDIS_adjusted.merge(EDIS_semi_features,on='participant_id')
SABRE_adjusted = SABRE_adjusted.merge(SABRE_semi_features,on='participant_id')
Insight_adjusted = Insight_adjusted.merge(Insight46_semi_features,on='participant_id')
HELIUS_adjusted = HELIUS_adjusted.merge(HELIUS_semi_features,on='participant_id')

In [ ]:
TOPMRI_adjusted = TOPMRI_adjusted.drop('Site', axis=1)
EDIS_adjusted = EDIS_adjusted.drop('Site', axis=1)
SABRE_adjusted = SABRE_adjusted.drop('Site', axis=1)
Insight_adjusted = Insight_adjusted.drop('Site', axis=1)
HELIUS_adjusted = HELIUS_adjusted.drop('Site', axis=1)

In [ ]:
TOPMRI_adjusted.to_csv('harm_results/covbat/topmri_covbat_5way.csv')
EDIS_adjusted.to_csv('harm_results/covbat/edis_covbat_5way.csv')
SABRE_adjusted.to_csv('harm_results/covbat/sabre_covbat_5way.csv')
Insight_adjusted.to_csv('harm_results/covbat/insight_covbat_5way.csv')
HELIUS_adjusted.to_csv('harm_results/covbat/helius_covbat_5way.csv')


In [ ]:
#HELIUS_adjusted.columns